In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 25 16:01:04 2023

@author: xinyuli
"""

#import cvxpy as cp 
from CongestionGame import congestion_game
from utils import *
from routing_game import *
from framework.game import *
from framework.utils import *
import copy
import math
import typing
import itertools
from tqdm import tqdm 

def random_policy_single_player(i, S, M):
    dic = {}
    Ai = ActionSet(i, [Action(i, value=j) for j in range(M)]) 

    for s in S:
        # Generate a list of random numbers that sum to 1
        random_numbers = [random.uniform(0, 1) for _ in range(M)]
        total_sum = sum(random_numbers)

        # Normalize the numbers to ensure they sum to 1
        normalized_numbers = [num / total_sum for num in random_numbers]

        # Print the list of random numbers that sum to 1
        cnt = 0
        for ai in Ai:
            dic[s,ai] = normalized_numbers[cnt]
            cnt += 1
    return dic


def generate_random_pi(I, S, A, M):
    single_player_policies = {}
    for i in I:
        single_player_policies[i] = random_policy_single_player(i, S, M)

    def generate_single_player_random_policy(i, s, a):
        return single_player_policies[i][s,a]
    pi = dict()
    for i in I:
        pi[i] = Policy(S, A[i], lambda s, a: generate_single_player_random_policy(i, s, a))
    pi = JointPolicy(pi)
    return pi


def modify_one_players_policy(i, pi, pi_prime, I, S, A):
    new_pi = dict()
    for j in I:
        if j != i:
            new_pi[j] = Policy(S, A[j], lambda s, ai: pi[j][s, ai])
        else:
            new_pi[j] = Policy(S, A[j], lambda s, ai: pi_prime[j][s, ai])
    new_pi = JointPolicy(new_pi)
    return new_pi

def gi(i, pi, pi_prime, phi, R, P, S, list_S, A, delta, mu):
    pi_a_i = pi[i]
    pi_minus_i = pi.minus(i)
    d_pi_a = construct_d_pi(i, pi_a_i, pi_minus_i, P, list_S, A, delta, mu)

    pi_b_i = pi_prime[i]
    d_pi_b = construct_d_pi(i, pi_b_i, pi_minus_i, P, list_S, A, delta, mu)

    return abs(sum((d_pi_b[idxs,idxa]-d_pi_a[idxs,idxa])*(phi[s][a]-R.get_reward(i, s, a))
        for idxs, s in enumerate(S) for idxa, a in enumerate(A)))


def g(pi, pi_prime,x, I, phi, R, P, S, list_S, A, delta, mu):
    max_val = 0
    for i in I:
        g_i = gi(i, pi, pi_prime, phi, R, P, S, list_S, A, delta, mu)
        if g_i > max_val:
            max_val = g_i
    return max_val -x

def find_best_potential_function_randomized(N, M, U, m, b, lambda_1, lambda_2, delta,
               K, tau, alpha_r, beta_r, common_interest, strategy_independent):
    game = congestion_game(N=N, M=M, U=U, m=m, b=b, lambda_1=lambda_1, lambda_2=lambda_2, delta=delta,
                                   common_interest=common_interest, strategy_independent_transitions=strategy_independent)
 
    I = game.I  # player set
    S = game.S  # state set
    A = game.A  # action profile set
    mu = game.mu  # initial state distribution
    P = game.P  # probability transition kernel
    R = game.R  # reward function
    delta = game.delta  # discount factor
    list_S = list(S)

    phi = {s: {a: game.reward_facility(s, a) for a in A} for s in S}

    def gamma_(n):
        return  1/(5* n) 

    def delta_(n):
        return (n)**0.45 



    T = 100
    X = 1
    for n in range(T+1):
        pi = generate_random_pi(I, S, A, M)
        pi_prime = generate_random_pi(I, S, A, M)
        g_val = g(pi, pi_prime, X, I, phi, R, P, S, list_S, A, delta, mu)
        X_new = X - gamma_(n+1) * (1 - delta_(n+1) * (g_val > 0))

        if X == 0:
            print("n =",n, ", X =", X, ", gamma =", gamma_(n+1),
                    ", g =", g_val )
            print("g:", g_val)
            #break
        if X_new < 0: X_new = 0
        
        X = X_new

    

We aim to solve the following semi-infinite programming problem:
$$ 
\begin{array}{lll}
&\min_x &x\\
&s.t. & x\geq |\sum_s \sum_a (d^{\mu}(\pi) -d^{\mu})(\pi_i', \pi_{-i}) (u_i - \phi)(s,a) )|, \quad \forall i \in I
\end{array}
$$
where $$\phi(s,a) = \sum_{j \in M}\sum_{k=1}^{\#(a,j)} c_j(k, s).$$
First define $g$ to be
$$
g(x, \pi, \pi') = \max_i \left|\sum_{s, a}  (d^\mu(\pi) - d^\mu(\pi_i', \pi_{-i}))(u_i - \phi)(s,a) \right| - x
$$
and let $h$ be
$$h(g) = \max\{0, g\}$$

According to the update rule (14) in Tadic et al,
$$
\begin{aligned}
X_{n+1} &= X_n - \gamma_{n+1} \nabla f(X_n) - \gamma_{n+1}  \delta_{n+1}h'(g) \nabla_x g(x, \pi, \pi') \\ 
\end{aligned}
$$
which is equivalent to 
$$ X_{n+1} = X_n - \gamma_{n+1}  + \gamma_{n+1}  \delta_{n+1} 1_{\{ g > 0\}}  $$


In the paper, it requires $\gamma_n = n^{-c}$ with $c\in(-0.5, 1]$.
Moreover, $\gamma_n >0, \sum \gamma_n = \infty, \sum \gamma_n^2 \delta_n^2 < \infty$

$\delta_n$ is an increase sequence with $\lim \delta_n = \infty$.

Our choice: $\gamma_n \propto \frac{1}{n}, \delta_n = n^{0.45}$.


In [2]:
N = 3
M = 2
U = 1
m = [2, 4]
b = [9, 16]
lambda_1 = 0.8
lambda_2 = 0.2
delta = 0.8
K = int(1e5)
tau = 1e-6
alpha_r = 0.5
beta_r = 1
common_interest =  False
strategy_independent = False
find_best_potential_function_randomized(N, M, U, m, b, lambda_1, lambda_2, delta,
               K, tau, alpha_r, beta_r, common_interest, strategy_independent)

n = 83 , X = 0 , gamma = 0.002380952380952381 , g = 1.8735013540549517e-16
g: 1.8735013540549517e-16
n = 91 , X = 0 , gamma = 0.002173913043478261 , g = 5.342948306008566e-16
g: 5.342948306008566e-16
n = 99 , X = 0 , gamma = 0.002 , g = 1.3877787807814457e-16
g: 1.3877787807814457e-16


In [3]:
N = 6
M = 2
U = 1
m = [2, 4]
b = [0, 0]
lambda_1 = 1
lambda_2 = 0
delta = 0.8
K = int(1e5)
tau = 1e-6
alpha_r = 0.5
beta_r = 1
common_interest =  False
strategy_independent = False
find_best_potential_function_randomized(N, M, U, m, b, lambda_1, lambda_2, delta,
               K, tau, alpha_r, beta_r, common_interest, strategy_independent)

n = 83 , X = 0 , gamma = 0.002380952380952381 , g = 2.05911676598447e-15
g: 2.05911676598447e-15
n = 91 , X = 0 , gamma = 0.002173913043478261 , g = 1.2212453270876722e-15
g: 1.2212453270876722e-15
n = 99 , X = 0 , gamma = 0.002 , g = 1.0234868508263162e-15
g: 1.0234868508263162e-15


## Appendix

Check
$$max_{i,j} | \sum_s\sum_a (d^\mu(\pi)-d^\mu(\pi_i’,\pi_j’,\pi_{-ij}))(u_i-\phi) |$$ 
is not very close to 0 to make sure the code does not have bug



In [4]:
def modify_two_players_policy(i, k, pi, pi_prime, I, S, A):
    new_pi = dict()
    for j in I:
        if j != i or j != k:
            new_pi[j] = Policy(S, A[j], lambda s, ai: pi[j][s, ai])
        else:
            new_pi[j] = Policy(S, A[j], lambda s, ai: pi_prime[j][s, ai])
    new_pi = JointPolicy(new_pi)
    return new_pi

In [8]:
game = congestion_game(N=N, M=M, U=U, m=m, b=b, lambda_1=lambda_1, lambda_2=lambda_2, delta=delta,
                                common_interest=common_interest, strategy_independent_transitions=strategy_independent)

I = game.I  # player set
S = game.S  # state set
A = game.A  # action profile set
mu = game.mu  # initial state distribution
P = game.P  # probability transition kernel
R = game.R  # reward function
delta = game.delta  # discount factor
list_S = list(S)

phi = {s: {a: game.reward_facility(s, a) for a in A} for s in S}


# i is the first player, k is the last player
for i in I:
    print(i)
    break
for k in I:
    print(k)

1
1
2
3
4
5
6


In [9]:
def generate_random_pi_dict(I, S, A, M):
    single_player_policies_a= {}
    for i in I:
        single_player_policies_a[i] = random_policy_single_player(i, S, M)

    return single_player_policies_a



def generate_single_player_random_policy(single_player_policies, i, s, a):
    return single_player_policies[i][s,a]



def create_pi_ik(single_player_policies_a, single_player_policies_b, i, k):
    """
    pi_i and pi_k are from policy b
    others from policy a
    """
    pi = dict()
    for j in I:
        if j !=i and j!=k:
            pi[j] = Policy(S, A[j], \
                lambda s, a: generate_single_player_random_policy(single_player_policies_a, j, s, a))
        else:
            pi[j] = Policy(S, A[j], \
                lambda s, a: generate_single_player_random_policy(single_player_policies_b, j, s, a))

    pi = JointPolicy(pi)
    return pi

In [10]:
single_player_policies_a = generate_random_pi_dict(I, S, A, M)
single_player_policies_b = generate_random_pi_dict(I, S, A, M)


In [11]:
def gi2(i, k, pi, pi_prime, phi, R, P, S, list_S, A, delta, mu):
    pi_ik_prime = create_pi_ik(single_player_policies_a, single_player_policies_b, i, k)
    pi_ik = create_pi_ik(single_player_policies_a, single_player_policies_a, i, k)

    pi_a_i = pi_ik[i]

    pi_minus_i = pi.minus(i)
    #d^mu(pi)
    d_pi_a = construct_d_pi(i, pi_a_i, pi_minus_i, P, list_S, A, delta, mu)

    
    pi_b_i = pi_ik_prime[i]
    pi_minus_i_b = pi_ik_prime.minus(i)
    #d^mu(pi'{i}, pi'{j}, pi{-i, -j})
    d_pi_b = construct_d_pi(i, pi_b_i, pi_minus_i_b, P, list_S, A, delta, mu)
    return abs(sum((d_pi_b[idxs,idxa]-d_pi_a[idxs,idxa])*(phi[s][a]-R.get_reward(i, s, a))
        for idxs, s in enumerate(S) for idxa, a in enumerate(A)))

 $$ g_{i} = | \sum_s\sum_a (d^\mu(\pi)-d^\mu(\pi_i',\pi_{-i}))(u_i-\phi) | $$

In [14]:
pi = generate_random_pi(I, S, A, M)
pi_prime = generate_random_pi(I, S, A, M)

gi(i, pi, pi_prime, phi, R, P, S, list_S, A, delta, mu)

1.3114509478384662e-15

 $$ g_{ik} = | \sum_s\sum_a (d^\mu(\pi)-d^\mu(\pi_i',\pi_k',\pi_{-ik}))(u_i-\phi) | $$

In [15]:
gi2(i,  k, pi, pi_prime, phi, R, P, S, list_S, A, delta, mu)

0.2231770513304578